In [1]:
import numpy as np
from mealpy.optimizer import Optimizer

In [2]:
class OriginalHHO(Optimizer):
    def __init__(self, epoch=10000, pop_size=100, **kwargs):
        """
        Args:
            epoch (int): maximum number of iterations, default = 10000
            pop_size (int): number of population size, default = 100
        """
        super().__init__(**kwargs)
        self.epoch = self.validator.check_int("epoch", epoch, [1, 100000])
        self.pop_size = self.validator.check_int("pop_size", pop_size, [10, 10000])
        self.set_parameters(["epoch", "pop_size"])
        self.sort_flag = False

    def evolve(self, epoch):
        """
        The main operations (equations) of algorithm. Inherit from Optimizer class
        Args:
            epoch (int): The current iteration
        """
        pop_new = []
        for idx in range(0, self.pop_size):
            # -1 < E0 < 1
            E0 = 2 * np.random.uniform() - 1
            # factor to show the decreasing energy of rabbit
            E = 2 * E0 * (1 - (epoch + 1) * 1.0 / self.epoch)
            J = 2 * (1 - np.random.uniform())

            # -------- Exploration phase Eq. (1) in paper -------------------
            if np.abs(E) >= 1:
                # Harris' hawks perch randomly based on 2 strategy:
                if np.random.rand() >= 0.5:  # perch based on other family members
                    X_rand = self.pop[np.random.randint(0, self.pop_size)][self.ID_POS].copy()
                    pos_new = X_rand - np.random.uniform() * np.abs(X_rand - 2 * np.random.uniform() * self.pop[idx][self.ID_POS])
                else:  # perch on a random tall tree (random site inside group's home range)
                    X_m = np.mean([x[self.ID_POS] for x in self.pop])
                    pos_new = (self.g_best[self.ID_POS] - X_m) - np.random.uniform() * \
                              (self.problem.lb + np.random.uniform() * (self.problem.ub - self.problem.lb))
                # pos_new = self.amend_position(pos_new, self.problem.lb, self.problem.ub)
                pop_new.append([pos_new, None])
            # -------- Exploitation phase -------------------
            else:
                # Attacking the rabbit using 4 strategies regarding the behavior of the rabbit
                # phase 1: ----- surprise pounce (seven kills) ----------
                # surprise pounce (seven kills): multiple, short rapid dives by different hawks
                if (np.random.rand() >= 0.5):
                    delta_X = self.g_best[self.ID_POS] - self.pop[idx][self.ID_POS]
                    if np.abs(E) >= 0.5:  # Hard besiege Eq. (6) in paper
                        pos_new = delta_X - E * np.abs(J * self.g_best[self.ID_POS] - self.pop[idx][self.ID_POS])
                    else:  # Soft besiege Eq. (4) in paper
                        pos_new = self.g_best[self.ID_POS] - E * np.abs(delta_X)
                    pos_new = self.amend_position(pos_new, self.problem.lb, self.problem.ub)
                    pop_new.append([pos_new, None])
                else:
                    LF_D = self.get_levy_flight_step(beta=1.5, multiplier=0.01, case=-1)
                    if np.abs(E) >= 0.5:  # Soft besiege Eq. (10) in paper
                        Y = self.g_best[self.ID_POS] - E * np.abs(J * self.g_best[self.ID_POS] - self.pop[idx][self.ID_POS])
                    else:  # Hard besiege Eq. (11) in paper
                        X_m = np.mean([x[self.ID_POS] for x in self.pop])
                        Y = self.g_best[self.ID_POS] - E * np.abs(J * self.g_best[self.ID_POS] - X_m)
                    pos_Y = self.amend_position(Y, self.problem.lb, self.problem.ub)
                    target_Y = self.get_target_wrapper(pos_Y)
                    Z = Y + np.random.uniform(self.problem.lb, self.problem.ub) * LF_D
                    pos_Z = self.amend_position(Z, self.problem.lb, self.problem.ub)
                    target_Z = self.get_target_wrapper(pos_Z)
                    if self.compare_agent([pos_Y, target_Y], self.pop[idx]):
                        pop_new.append([pos_Y, target_Y])
                        continue
                    if self.compare_agent([pos_Z, target_Z], self.pop[idx]):
                        pop_new.append([pos_Z, target_Z])
                        continue
                    pop_new.append(self.pop[idx].copy())
        if self.mode not in self.AVAILABLE_MODES:
            for idx, agent in enumerate(pop_new):
                pop_new[idx][self.ID_TAR] = self.get_target_wrapper(agent[self.ID_POS])
        else:
            pop_new = self.update_target_wrapper_population(pop_new)
        self.pop = self.greedy_selection_population(self.pop, pop_new)

In [3]:
def fitness_function(solution):
    return 50*solution[0]+100*solution[1]

problem_dict1 = {
    "fit_func": fitness_function,
    "lb": [0,0],
    "ub": [500,500],
    "minmax": "max",
}

epoch = 10
pop_size = 50

model = OriginalHHO(epoch, pop_size)
best_position, best_fitness = model.solve(problem_dict1)

print(f"Solution: {best_position}, Fitness: {best_fitness}")

2023/11/04 02:30:51 PM, INFO, __main__.OriginalHHO: Solving single objective optimization problem.
2023/11/04 02:30:51 PM, INFO, __main__.OriginalHHO: >Problem: P, Epoch: 1, Current best: 75000.0, Global best: 75000.0, Runtime: 0.01798 seconds
2023/11/04 02:30:51 PM, INFO, __main__.OriginalHHO: >Problem: P, Epoch: 2, Current best: 75000.0, Global best: 75000.0, Runtime: 0.01922 seconds
2023/11/04 02:30:51 PM, INFO, __main__.OriginalHHO: >Problem: P, Epoch: 3, Current best: 75000.0, Global best: 75000.0, Runtime: 0.01989 seconds
2023/11/04 02:30:51 PM, INFO, __main__.OriginalHHO: >Problem: P, Epoch: 4, Current best: 75000.0, Global best: 75000.0, Runtime: 0.01660 seconds
2023/11/04 02:30:51 PM, INFO, __main__.OriginalHHO: >Problem: P, Epoch: 5, Current best: 75000.0, Global best: 75000.0, Runtime: 0.01761 seconds
2023/11/04 02:30:51 PM, INFO, __main__.OriginalHHO: >Problem: P, Epoch: 6, Current best: 75000.0, Global best: 75000.0, Runtime: 0.02372 seconds
2023/11/04 02:30:51 PM, INFO, _

Solution: [500. 500.], Fitness: 75000.0


In [4]:
import pulp
import tkinter as tk
from tkinter import ttk

In [5]:
# Create a Linear Programming problem
lp_problem = pulp.LpProblem("LP_Problem", pulp.LpMinimize)

# Define decision variables
x = pulp.LpVariable("x", lowBound=0)
y = pulp.LpVariable("y", lowBound=0)

# Define the objective function to minimize
objective_function = 200 * x + 500 * y
lp_problem += objective_function, "Objective"

# Define constraints
constraint1 = x + 2 * y >= 10
constraint2 = 3 * x + 4 * y <= 24
lp_problem += constraint1, "Constraint 1"
lp_problem += constraint2, "Constraint 2"

# Solve the LP problem
lp_problem.solve()

# Print the status of the solution
print("Status:", pulp.LpStatus[lp_problem.status])

# Print the optimal values of decision variables
optimal_x = x.varValue
optimal_y = y.varValue
print("Optimal Solution:")
print("x =", optimal_x)
print("y =", optimal_y)

# Print the optimal objective value
optimal_value = pulp.value(lp_problem.objective)
print("Optimal Objective Value =", optimal_value)


Status: Optimal
Optimal Solution:
x = 4.0
y = 3.0
Optimal Objective Value = 2300.0


In [6]:
# Create a Linear Programming problem
lp_problem = pulp.LpProblem("LP_Problem", pulp.LpMaximize)  # Change to maximize

# Define decision variables
x = pulp.LpVariable("x", lowBound=0)
y = pulp.LpVariable("y", lowBound=0)

# Define the objective function to maximize
objective_function = 200 * x + 500 * y
lp_problem += objective_function, "Objective"

# Define constraints
constraint1 = x + 2 * y >= 10
constraint2 = 3 * x + 4 * y <= 24
lp_problem += constraint1, "Constraint 1"
lp_problem += constraint2, "Constraint 2"

# Solve the LP problem
lp_problem.solve()

# Print the status of the solution
print("Status:", pulp.LpStatus[lp_problem.status])

# Print the optimal values of decision variables
optimal_x = x.varValue
optimal_y = y.varValue
print("Optimal Solution:")
print("x =", optimal_x)
print("y =", optimal_y)

# Print the optimal objective value
optimal_value = pulp.value(lp_problem.objective)
print("Optimal Objective Value =", optimal_value)

Status: Optimal
Optimal Solution:
x = 0.0
y = 6.0
Optimal Objective Value = 3000.0


In [7]:
# Create a Linear Programming problem
lp_problem = pulp.LpProblem("LP_Problem", pulp.LpMaximize)  # Change to maximize if needed

# Define decision variables
variables = {}  # Create a dictionary to store decision variables
variables['x'] = pulp.LpVariable('x', lowBound=0)
variables['y'] = pulp.LpVariable('y', lowBound=0)
# Add more decision variables as needed, e.g., variables['z'] = pulp.LpVariable('z', lowBound=0)

# Define the objective function (modify coefficients as needed)
objective_function = 200 * variables['x'] + 500 * variables['y']  # Modify coefficients
# Add more terms for additional variables, e.g., + 100 * variables['z']
lp_problem += objective_function, "Objective"

# Define constraints
constraints = {}  # Create a dictionary to store constraints
constraints['constraint1'] = variables['x'] + 2 * variables['y'] >= 10
constraints['constraint2'] = 3 * variables['x'] + 4 * variables['y'] <= 24
# Add more constraints as needed, e.g., constraints['constraint3'] = ...

# Add constraints to the LP problem
for constraint_name, constraint in constraints.items():
    lp_problem += constraint, constraint_name

# Solve the LP problem
lp_problem.solve()

# Print the status of the solution
print("Status:", pulp.LpStatus[lp_problem.status])

# Print the optimal values of decision variables
optimal_values = {var: var_varValue for var, var_varValue in variables.items()}
print("Optimal Solution:")
for var, var_varValue in optimal_values.items():
    print(f"{var} =", var_varValue)

# Print the optimal objective value
optimal_value = pulp.value(lp_problem.objective)
print("Optimal Objective Value =", optimal_value)

Status: Optimal
Optimal Solution:
x = x
y = y
Optimal Objective Value = 3000.0


In [8]:
import numpy as np
from scipy.optimize import minimize

# Define the objective function to be minimized
def objective_function(x):
    x1, x2 = x
    return x1**2 + x2**2

# Define the inequality constraint function
def constraint1(x):
    x1, x2 = x
    return 1 - (x1 + x2)

# Define the initial guess for the decision variables
initial_guess = [0.5, 0.5]

# Define the bounds for decision variables (x1 and x2)
bounds = ((0, None), (0, None))

# Solve the NLP problem using SciPy's minimize function
result = minimize(objective_function, initial_guess, method='SLSQP', bounds=bounds, constraints={'type': 'ineq', 'fun': constraint1})

# Extract the optimization results
optimal_solution = result.x
optimal_objective_value = result.fun

# Print the results
print("Optimal Solution (x1, x2):", optimal_solution)
print("Optimal Objective Value:", optimal_objective_value)


Optimal Solution (x1, x2): [1.11022302e-16 1.11022302e-16]
Optimal Objective Value: 2.465190328815662e-32


In [ ]:
import numpy as np
from scipy.optimize import minimize

# Define the objective function to be minimized
def objective_function(x):
    # Replace this with your custom objective function
    return -your_objective_function_here(x)

# Define the constraint function
def constraint_function(x):
    # Replace this with your custom constraint function
    return your_constraint_here(x)

# Define the initial guess for the decision variables
initial_guess = [0, 0]  # Provide your initial guess

# Define the bounds for decision variables (x1 and x2)
bounds = ((0, None), (0, None))  # Adjust the bounds as needed

# Define the constraint type ('eq' for equality or 'ineq' for inequality)
constraint_type = 'eq'  # Change to 'ineq' for inequality constraints

# Create a dictionary for the constraint
constraints = {'type': constraint_type, 'fun': constraint_function}

# Solve the NLP problem using SciPy's minimize function
result = minimize(objective_function, initial_guess, method='SLSQP', bounds=bounds, constraints=constraints)

# Extract the optimization results
optimal_solution = result.x
optimal_objective_value = -result.fun  # Restore the negative sign

# Print the results
print("Optimal Solution (x1, x2):", optimal_solution)
print("Optimal Objective Value:", optimal_objective_value)


In [12]:
import numpy as np
from scipy.optimize import minimize

# Define the objective function to be minimized
def objective_function(x):
    x1, x2 = x
    return -(4 * x1 - 0.1 * x1**2 + 5 * x2 - 0.2 * x2**2)  # We use the negative sign because we're minimizing

# Define the equality constraint function
def constraint1(x):
    x1, x2 = x
    return x1 + 2 * x2 - 40

# Define the initial guess for the decision variables
initial_guess = [0, 0]

# Define the bounds for decision variables (x1 and x2)
bounds = ((0, None), (0, None))

# Create a dictionary for the equality constraint
constraints = {'type': 'eq', 'fun': constraint1}

# Solve the NLP problem using SciPy's minimize function
result = minimize(objective_function, initial_guess, method='SLSQP', bounds=bounds, constraints=constraints)

# Extract the optimization results
optimal_solution = result.x
optimal_objective_value = -result.fun  # Restore the negative sign

# Print the results
print("Optimal Solution (x1, x2):", optimal_solution)
print("Optimal Objective Value:", optimal_objective_value)


Optimal Solution (x1, x2): [18.33332368 10.83333816]
Optimal Objective Value: 70.41666666665267
